In a nutshell, this notebook implemented a Gaussian process regression with a NNGP kernel to return predicted Y_test given X_test. The followings are to explain how the implementation allows the computation of the GP regression and the kernel, in a reverse-engineering perspective.

First of all, the Gaussian Process regression model was based on GPflow, and it is to estimate a GP $p(y^* | x, y, x^*) \sim N(\bar{\mu},\bar{K})$ according to the equation (ref.4):

$$
\bar{\mu} = K^L_{x^*, D} (K^L_{D, D} + \sigma^2 \mathbb{I}_n)^{-1} t 
$$
and 
$$
\bar{K}=K^L_{x^*,x^*} - K^L_{x^*,D}(K^L_{D, D} + \sigma^2 \mathbb{I}_n)^{-1} K^{L,T}_{x^*, D} 
$$

where $\bar{\mu}$ is the mean of predicted Y, $\bar{K}$ represents the variance of the prediction. The model computes the Cholesky decompositionthe of $K^L$ as part of the algorithm for finding $\bar{\mu}$ and $\bar{K}$, using the functions defined under the class GaussianProcessRegression.


Next, in order to get the $K^L$ for the model, a NNGP kernel was implemented. The key function under the class NNGPKernel is k_full(). It first computes $K^l$ that is the covariance of post-activation at given pre-activation variance and correlation. It then returns a fully stacked $K^L$ over all the layers in the neural network. The implementation was based on the euqation (ref.2):

$$
K^l(x,x') = \sigma^2_b + \sigma^2_w \mathbb{E}_{z^{l-1}_i \sim GP(0, K^{l-1})}[\phi(z^{l-1}_i (x))\phi(z^{l-1}_i (x')]
$$

where the expectation of $z_i^{(l-1)}$ was computed via linear interpolation in the NNGP grid using the interp functions. 

To describe the implementation of k_full() in peudocodes.

k_full (): 
    
    Normalize input to unit variance or to fixed point variance
    
    For each hidden layer l:
        q_ab = interp.interp_lin_2d(...) 
        q_ab = self.weight_var * q_ab + self.bias_var
        
    q_ab_all = parallelly_stack(q_ab) 
    
    return q_ab_all 



In addition, the following tensorboard graph captures the workflow of k_full() .
<img src="kfull.png">


The following tensorboard graph captures the workflow of  the Gaussian process regression.
<img src="GPR.png">